In [1]:
# Bimbo XGBoost R script LB:0.457
# Author: Bohdan Pavlyshenko e-mail: b.pavlyshenko@gmail.com
#
# One of the main ideas of the script is to use lagged values for target variable.
# Due to memory limits 8GB in the Kaggle cloud R server, we use only one week lag for target variable.
# If to start this script on the computer with 32GB and XGBoost parameter nrounds=150, we can use lagged values
# of target variable for 5 weeks which gives us the leaderbord score 0.45728

library(data.table)
library(xgboost)

train=fread('./input-data/train.csv',select = c("Semana",'Cliente_ID', 'Producto_ID', 'Agencia_ID', 'Ruta_SAK', 'Demanda_uni_equil'))
test=fread('./input-data/test.csv',select = c("Semana",'id','Cliente_ID', 'Producto_ID', 'Agencia_ID', 'Ruta_SAK'))

Read 74180464 rows and 6 (of 11) columns from 2.980 GB file in 00:00:23


In [2]:
# Cut the train set to 8 and 9 weeks (Semana) for using only one week lags for target variable.
# If you have enough memory, you can set up condition Semana>3 on the next row for using lagged values of target variable for 5 weeks. 
train=train[Semana>3,]

train$id=0
train[,target:=Demanda_uni_equil]
train[,Demanda_uni_equil:=NULL]
train[,tst:=0]
test$target=0
test[,tst:=1]
data=rbind(train,test)
rm(test)  
rm(train)

# Creating features for one week lagged values of target variable 
data1<-data[,.(Semana=Semana+1,Cliente_ID,Producto_ID,target)]
data=merge(data,data1[Semana>8,.(targetl1=mean(target)), by=.(Semana,Cliente_ID,Producto_ID)],all.x=T, by=c("Semana","Cliente_ID","Producto_ID"))

# If you have enough memory, you can set up value TRUE in the following condition to make it possible to use lagged values of target variable 
# for lagged 2-5 weeks as additional features 
if(TRUE){
  data1<-data[,.(Semana=Semana+2,Cliente_ID,Producto_ID,target)]
  data=merge(data,data1[Semana>8,.(targetl2=mean(target)), by=.(Semana,Cliente_ID,Producto_ID)],all.x=T, by=c("Semana","Cliente_ID","Producto_ID"))
  data1<-data[,.(Semana=Semana+3,Cliente_ID,Producto_ID,target)]
  data=merge(data,data1[Semana>8,.(targetl3=mean(target)), by=.(Semana,Cliente_ID,Producto_ID)],all.x=T, by=c("Semana","Cliente_ID","Producto_ID"))
  data1<-data[,.(Semana=Semana+4,Cliente_ID,Producto_ID,target)]
  data=merge(data,data1[Semana>8,.(targetl4=mean(target)), by=.(Semana,Cliente_ID,Producto_ID)],all.x=T, by=c("Semana","Cliente_ID","Producto_ID"))
  data1<-data[,.(Semana=Semana+5,Cliente_ID,Producto_ID,target)]
  data=merge(data,data1[Semana>8,.(targetl5=mean(target)), by=.(Semana,Cliente_ID,Producto_ID)],all.x=T, by=c("Semana","Cliente_ID","Producto_ID"))
}

rm(data1)
data=data[Semana>8,]


ERROR: Error in rbindlist(l, use.names, fill, idcol): Item 2 of list input is not a data.frame, data.table or list


          Semana Agencia_ID Ruta_SAK Cliente_ID Producto_ID Demanda_uni_equil
       1:      4       1110     3301      15766         325                 1
       2:      4       1110     3301      15766         328                 1
       3:      4       1110     3301      15766        1212                 4
       4:      4       1110     3301      15766        1216                 2
       5:      4       1110     3301      15766        1238                 3
      ---                                                                    
63015253:      9      25759     5517    4528866       32873                 4
63015254:      9      25759     5517    4528866       34226                 4
63015255:      9      25759     5517    4528866       45112                 4
63015256:      9      25759     5517    4547943       40217                 0
63015257:      9      25759     5517    4708097       43159                 1
          id target
       1:  0      1
       2:  0      1
    

ERROR: Error in rbindlist(l, use.names, fill, idcol): Item 2 of list input is not a data.frame, data.table or list


          Semana Agencia_ID Ruta_SAK Cliente_ID Producto_ID id target
       1:      4       1110     3301      15766         325  0      1
       2:      4       1110     3301      15766         328  0      1
       3:      4       1110     3301      15766        1212  0      4
       4:      4       1110     3301      15766        1216  0      2
       5:      4       1110     3301      15766        1238  0      3
      ---                                                            
63015253:      9      25759     5517    4528866       32873  0      4
63015254:      9      25759     5517    4528866       34226  0      4
63015255:      9      25759     5517    4528866       45112  0      4
63015256:      9      25759     5517    4547943       40217  0      0
63015257:      9      25759     5517    4708097       43159  0      1

ERROR: Error in rbindlist(l, use.names, fill, idcol): Item 2 of list input is not a data.frame, data.table or list


          Semana Agencia_ID Ruta_SAK Cliente_ID Producto_ID id target tst
       1:      4       1110     3301      15766         325  0      1   0
       2:      4       1110     3301      15766         328  0      1   0
       3:      4       1110     3301      15766        1212  0      4   0
       4:      4       1110     3301      15766        1216  0      2   0
       5:      4       1110     3301      15766        1238  0      3   0
      ---                                                                
63015253:      9      25759     5517    4528866       32873  0      4   0
63015254:      9      25759     5517    4528866       34226  0      4   0
63015255:      9      25759     5517    4528866       45112  0      4   0
63015256:      9      25759     5517    4547943       40217  0      0   0
63015257:      9      25759     5517    4708097       43159  0      1   0

ERROR: Error in rbindlist(l, use.names, fill, idcol): Item 2 of list input is not a data.frame, data.table or list


              id Semana Agencia_ID Ruta_SAK Cliente_ID Producto_ID target tst
      1:       0     11       4037     2209    4639078       35305      0   1
      2:       1     11       2237     1226    4705135        1238      0   1
      3:       2     10       2045     2831    4549769       32940      0   1
      4:       3     11       1227     4448    4717855       43066      0   1
      5:       4     11       1219     1130     966351        1277      0   1
     ---                                                                     
6999247: 6999246     11       2057     1153    4379638        1232      0   1
6999248: 6999247     10       1334     2008     970421       43069      0   1
6999249: 6999248     11       1622     2869     192749       30532      0   1
6999250: 6999249     11       1636     4401     286071       35107      0   1
6999251: 6999250     11       1625     1259     978760        1232      0   1

In [ ]:
# Creating frequency features for some factor variables
nAgencia_ID=data[,.(nAgencia_ID=.N),by=.(Agencia_ID,Semana)]
nAgencia_ID=nAgencia_ID[,.(nAgencia_ID=mean(nAgencia_ID,na.rm=T)),by=Agencia_ID]
data=merge(data,nAgencia_ID,by='Agencia_ID',all.x=T)
nRuta_SAK=data[,.(nRuta_SAK=.N),by=.(Ruta_SAK,Semana)]
nRuta_SAK=nRuta_SAK[,.(nRuta_SAK=mean(nRuta_SAK,na.rm=T)),by=Ruta_SAK]
data=merge(data,nRuta_SAK,by='Ruta_SAK',all.x=T)
nCliente_ID=data[,.(nCliente_ID=.N),by=.(Cliente_ID,Semana)]
nCliente_ID=nCliente_ID[,.(nCliente_ID=mean(nCliente_ID,na.rm=T)),by=Cliente_ID]
data=merge(data,nCliente_ID,by='Cliente_ID',all.x=T)
nProducto_ID=data[,.(nProducto_ID=.N),by=.(Producto_ID,Semana)]
nProducto_ID=nProducto_ID[,.(nProducto_ID=mean(nProducto_ID,na.rm=T)),by=Producto_ID]
data=merge(data,nProducto_ID,by='Producto_ID',all.x=T)
data$target=log(data$target+1)
data_train=data[tst==0,]
data_test=data[tst==1,]

features=names(data_train)[!(names(data_train) %in% c('id',"target",'tst'))] 

rm(data)

In [ ]:
wltst=sample(nrow(data_train),30000)  

dval<-xgb.DMatrix(data=data.matrix(data_train[wltst,features,with=FALSE]),
                  label=data.matrix(data_train[wltst,target]),missing=NA)
watchlist<-list(dval=dval)

clf <- xgb.train(params=list(  objective="reg:linear", 
                               booster = "gbtree",
                               eta=0.1, 
                               max_depth=10, 
                               subsample=0.85,
                               colsample_bytree=0.7) ,
                 data = xgb.DMatrix(data=data.matrix(data_train[-wltst,features,with=FALSE]),
                                    label=data.matrix(data_train[-wltst,target]),missing=NA), 
                 nrounds = 300, 
                 verbose = 1,
                 print_every_n=5,
                 early_stopping_rounds    = 10,
                 watchlist           = watchlist,
                 maximize            = FALSE,
                 eval_metric='rmse'
)

# Make prediction for the 10th week
data_test1=data_test[Semana==10,]
pred<-predict(clf,xgb.DMatrix(data.matrix(data_test1[,features,with=FALSE]),missing=NA))
res=exp(round(pred,5))-1


In [ ]:
# Create lagged values of target variable which will be used as a feature for the 11th week prediction 
data_test_lag1=data_test1[,.(Cliente_ID,Producto_ID)]
data_test_lag1$targetl1=res
data_test_lag1=data_test_lag1[,.(targetl1=mean(targetl1)), by=.(Cliente_ID,Producto_ID)]

results=data.frame(id=data_test1$id,Demanda_uni_equil=res)

data_test2=data_test[Semana==11,]
data_test2[,targetl1:=NULL]

In [ ]:
# Merge lagged values of target variable to test the set for the 11th week
data_test2=merge(data_test2,data_test_lag1,all.x=T,by=c('Cliente_ID','Producto_ID'))
pred<-predict(clf,xgb.DMatrix(data.matrix(data_test2[,features,with=FALSE]),missing=NA))
res=exp(round(pred,5))-1
res.df=data.frame(id=data_test2$id,Demanda_uni_equil=res)
results=rbind(results, res.df)

results[results[,2]<0,2]=0
results[,2]=round(results[,2],1)
results[,1]=as.integer(results[,1])
class(results[,1])='int32'
options(digits=18)
write.csv(results,file='./Submissions/results1.csv',row.names=F)